# EDA

In this EDA, the study dataset are examined in more detail. The goal is to understand the data and the structure of it better.


In [188]:
import pandas as pd
import os
import plotly.express as px
import re
import numpy as np
import plotly.graph_objs as go
from IPython.display import display
from IPython.display import Markdown as md
import plotly.io as pio
from pandas_profiling import ProfileReport
from globalvars import *
from functions import *
import kaleido

### Load Data

In [190]:
## Import dataframe
filelocation = 'data/DataClean'
df = pd.read_feather(filelocation)

## load config file
config = pd.read_excel('config/config.xlsx',sheet_name='fragecodes')
invites_month = pd.read_excel('config/config.xlsx',sheet_name='invites')

In [191]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230436 entries, 0 to 230435
Data columns (total 41 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   participant_id       230436 non-null  int64         
 1   u_date               230436 non-null  datetime64[ns]
 2   Kommentar            62178 non-null   object        
 3   wime_personal        79167 non-null   float64       
 4   wime_komfort         179565 non-null  float64       
 5   wime_sauberkeit      182445 non-null  float64       
 6   wime_puenktlich      183071 non-null  float64       
 7   wime_platzangebot    183866 non-null  float64       
 8   wime_gesamtzuf       191117 non-null  float64       
 9   wime_preis_leistung  214913 non-null  float64       
 10  wime_fahrplan        222078 non-null  float64       
 11  wime_oes_fahrt       175091 non-null  float64       
 12  S_sprache            230436 non-null  category      
 13  S_alter       

In [192]:
df[df['u_fahrausweis']=='Normales Billett'].sort_values('u_date',ascending=True)

,participant_id,u_date,Kommentar,wime_personal,wime_komfort,wime_sauberkeit,wime_puenktlich,wime_platzangebot,wime_gesamtzuf,wime_preis_leistung,...,ft_vm_kurz,ft_zielort_uic,fg_abfahrt,fg_ankunft,fg_startort_uic,fg_zielort_uic,fg_startort,fg_zielort,ft_startort,ft_zielort
110631,422617,2020-11-03,None,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaT,NaT,NaN,NaN,Luzern,Zürich Flughafen,Luzern,Zürich Flughafen
110725,422738,2020-11-03,None,NaN,NaN,NaN,NaN,NaN,NaN,25.0,...,NaN,NaN,2022-11-25 14:36:00,2022-11-25 15:06:00,8581537,8581532,"Dägerlen, Berg","Hettlingen, Bahnhof",NaN,NaN
110724,422736,2020-11-03,None,NaN,NaN,NaN,NaN,NaN,NaN,75.0,...,NaN,NaN,2022-11-25 10:05:00,2022-11-25 10:25:00,8579901,8507110,"Bümpliz, Statthalterstrasse","Bern, Zytglogge",NaN,NaN
110723,422735,2020-11-03,None,100.0,75.0,75.0,75.0,100.0,75.0,50.0,...,IR,8503000,2022-11-25 07:05:00,2022-11-25 08:06:00,8508008,8503000,Herzogenbuchsee,Zürich HB,Herzogenbuchsee,Zürich HB
110722,422734,2020-11-03,None,NaN,NaN,NaN,NaN,NaN,NaN,75.0,...,NaN,NaN,NaT,NaT,NaN,NaN,Zürich HB,"Luzern, Würzenbach",Zürich HB,"Luzern, Würzenbach"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230171,589587,2022-11-14,None,100.0,100.0,100.0,100.0,100.0,100.0,75.0,...,IC,8506302,2022-11-25 12:53:00,2022-11-25 13:52:00,8503016,8506302,Zürich Flughafen,St. Gallen,Zürich Flughafen,St. Gallen
230170,589585,2022-11-14,None,100.0,75.0,75.0,75.0,50.0,75.0,75.0,...,IC,8503000,2022-11-25 07:18:00,2022-11-25 08:28:00,8503016,8500010,Zürich Flughafen,Basel SBB,Zürich Flughafen,Zürich HB
230290,589750,2022-11-14,None,100.0,75.0,100.0,25.0,75.0,100.0,100.0,...,IC,8500010,2022-11-25 07:26:00,2022-11-25 09:22:00,8508202,8589286,Konolfingen,"Allschwil, Kirche",Bern,Basel SBB
230276,589731,2022-11-14,None,NaN,75.0,100.0,0.0,100.0,75.0,75.0,...,IC,8506302,2022-11-25 08:33:00,2022-11-25 09:32:00,8503000,8506302,Zürich HB,St. Gallen,Zürich HB,St. Gallen


## EDA

Possiblity to filter on a specific timerange. The filter is applied the whole EDA.

In [193]:
startdate = '2019-01-01'
enddate = '2022-10-31'

# Apply time selection for all DataFrames
df= df[(df['u_date'] > startdate) & (df['u_date'] < enddate)]

### Overview

#### General DataFrame Overview

In [194]:
df = df.sort_values("u_date",ascending=False)

In [195]:
df.head(5)

,participant_id,u_date,Kommentar,wime_personal,wime_komfort,wime_sauberkeit,wime_puenktlich,wime_platzangebot,wime_gesamtzuf,wime_preis_leistung,...,ft_vm_kurz,ft_zielort_uic,fg_abfahrt,fg_ankunft,fg_startort_uic,fg_zielort_uic,fg_startort,fg_zielort,ft_startort,ft_zielort
225817,583530,2022-10-30,Klimaanlage / Heizung wechselte leider ständig...,NaN,75.0,100.0,100.0,75.0,75.0,75.0,...,IC,8503000,2022-11-25 10:03:00,2022-11-25 13:27:00,8505479,8506000,Lodano,Winterthur,Bellinzona,Zürich HB
226068,583884,2022-10-30,None,NaN,NaN,NaN,NaN,NaN,NaN,75.0,...,NaN,NaN,NaT,NaT,NaN,NaN,Zofingen,"Rodersdorf, Station",Zofingen,"Rodersdorf, Station"
225782,583485,2022-10-30,None,NaN,100.0,75.0,100.0,100.0,100.0,100.0,...,S,8503301,2022-11-25 09:25:00,2022-11-25 09:33:00,8503303,8503301,Illnau,Pfäffikon ZH,Illnau,Pfäffikon ZH
226048,583862,2022-10-30,Am Morgen des gleichen Tages fuhren wir mit ei...,NaN,NaN,NaN,NaN,NaN,NaN,75.0,...,NaN,NaN,NaT,NaT,NaN,NaN,Chur,"Rafz, Friedhof",Chur,"Rafz, Friedhof"
226052,583866,2022-10-30,Wir haben 4 Veloplätze reserviert. Die Zuteilu...,NaN,NaN,NaN,NaN,NaN,NaN,75.0,...,NaN,NaN,NaT,NaT,NaN,NaN,Lugano,Zürich HB,Lugano,Zürich HB


In [196]:
df.shape

(226081, 41)

In [197]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 226081 entries, 225817 to 913
Data columns (total 41 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   participant_id       226081 non-null  int64         
 1   u_date               226081 non-null  datetime64[ns]
 2   Kommentar            61035 non-null   object        
 3   wime_personal        77322 non-null   float64       
 4   wime_komfort         176141 non-null  float64       
 5   wime_sauberkeit      178969 non-null  float64       
 6   wime_puenktlich      179579 non-null  float64       
 7   wime_platzangebot    180363 non-null  float64       
 8   wime_gesamtzuf       187619 non-null  float64       
 9   wime_preis_leistung  210793 non-null  float64       
 10  wime_fahrplan        217910 non-null  float64       
 11  wime_oes_fahrt       171773 non-null  float64       
 12  S_sprache            226081 non-null  category      
 13  S_alter     

In [198]:
## Add additional date columns for easier vizualization
df.insert(loc=2, column = 'year', value=df['u_date'].dt.year) #create additional year col for viz
df.insert(loc=3, column = 'month', value=df['u_date'].dt.month) #create additional month col for viz
df['yearmonth'] = pd.to_datetime(df[['year', 'month']].assign(DAY=1)) #create additional yearmonth col for viz

### TextData

In [364]:

df_text = df.dropna(subset=["Kommentar"])

#### Number of comments per month

In [365]:
df_text_counts = pd.DataFrame(df_text.groupby(['yearmonth'])['participant_id'].count()).reset_index()
df_participantstotals_counts = pd.DataFrame(df.groupby(['yearmonth'])['participant_id'].count()).reset_index()

In [366]:
# Merge Dataframes
df_counts_month = df_text_counts.merge(df_participantstotals_counts, on='yearmonth', how='left')
df_counts_month = df_counts_month.merge(invites_month, on='yearmonth', how='left') # join aount of invites
df_counts_month.rename(columns = {'participant_id_x':'Anzahl Befragte mit Kommentar','participant_id_y':'Anzahl Befragte'}, inplace = True)
df_counts_month['Kommentar Ratio in %'] = (df_counts_month['Anzahl Befragte mit Kommentar']/df_counts_month['Anzahl Befragte'])*100
df_counts_month['Rücklauf Ratio in %'] = (df_counts_month['Anzahl Befragte']/df_counts_month['Anzahl Einladungen'])*100

In [ ]:
df_counts_month.describe()

,Anzahl Befragte mit Kommentar,Anzahl Befragte,Anzahl Einladungen,Kommentar Ratio in %,Rücklauf Ratio in %
count,44.000000,44.000000,44.000000,44.000000,44.000000
mean,1387.159091,5138.204545,75714.863636,26.786723,6.743027
std,495.655823,1679.551554,20271.876486,2.545399,0.950641
min,637.000000,2314.000000,42605.000000,22.506454,4.752712
25%,999.750000,4018.750000,60434.000000,24.669403,6.196167
50%,1328.500000,5111.500000,67946.500000,26.288042,6.726789
75%,1717.500000,5903.500000,89381.250000,28.541672,7.096513
max,2698.000000,9898.000000,122993.000000,31.907954,9.891194


: 

In [367]:
len(df)

226081

In [368]:
## plot settings
color_discrete_sequence = color_discrete_sequence #imported from globalvars.py
template = template ##imported from globalvars.py

In [415]:
fig = px.line(df_counts_month,
              x="yearmonth",
              y=['Anzahl Befragte mit Kommentar', 'Anzahl Befragte','Anzahl Einladungen'],
              color_discrete_sequence=color_discrete_sequence,
              template=template)

fig.update_layout(
    title="Anzahl Einladungen, Anzahl Befragungen & Anzahl Befragte mit Kommentar",
    yaxis_title='',
    xaxis_title='',
    width=1200, 
    height=600,
    legend_title="")

fig.update_traces(line_width=2)
fig.update_traces(patch={"line": {"dash": 'dot'}}, selector={"legendgroup": "Anzahl Einladungen"})

fig.update_yaxes(type="log") 

fig.add_vrect(
    x0="2020-03-01",
    x1="2020-05-31",
    line_width=0,
    fillcolor="#0B1F26",
    opacity=0.2,
    annotation_text="Pause",
    annotation_position="top left",
    annotation_font_size=10,
    annotation_font_color="#0B1F26",
)

fig.show()

pio.write_image(fig,"images/fig-anteil_befragte_und_anzahl_kommentare.svg") #save fig as svg

In [425]:
fig = px.line(df_counts_month,
              x="yearmonth",
              y='Kommentar Ratio in %',
              color_discrete_sequence=color_discrete_sequence,
              template=template)
fig.update_layout(
    title="Anteil Befragte mit Kommentar",
    yaxis_title='Anteil in %',
    xaxis_title='Monat',
    width=1200, 
    height=600,
    )
fig.update_traces(line_width=2)
#fig.update_yaxes(range=(0,40))


# show legend for singel line plots
fig['data'][0]['showlegend'] = True
fig['data'][0]['name'] = 'Anteil Kommentare in %'

fig.update_yaxes(range=(21,35))

fig.show()

pio.write_image(fig,"images/fig-anteil-befragte-mit-kommentar.svg") #save fig as svg

In [393]:
df_counts_month.describe()

,Anzahl Befragte mit Kommentar,Anzahl Befragte,Anzahl Einladungen,Kommentar Ratio in %,Rücklauf Ratio in %
count,44.000000,44.000000,44.000000,44.000000,44.000000
mean,1387.159091,5138.204545,75714.863636,26.786723,6.743027
std,495.655823,1679.551554,20271.876486,2.545399,0.950641
min,637.000000,2314.000000,42605.000000,22.506454,4.752712
25%,999.750000,4018.750000,60434.000000,24.669403,6.196167
50%,1328.500000,5111.500000,67946.500000,26.288042,6.726789
75%,1717.500000,5903.500000,89381.250000,28.541672,7.096513
max,2698.000000,9898.000000,122993.000000,31.907954,9.891194


#### Minimal, maximal and average length of Kommentar

In [394]:
## Add basic text features
df_text["Kommentar"] = remove_redundant_whitespaces(df_text["Kommentar"]) #note: imported function "remove_redundant_whitespaces"
df_text = add_basic_textfeatures(df_text,"Kommentar")

In [395]:
df_text_average_character_median_per_month = pd.DataFrame(df_text.groupby(['yearmonth'])['Kommentar_Character'].median()).reset_index()
df_text_average_character_mean_per_month = pd.DataFrame(df_text.groupby(['yearmonth'])['Kommentar_Character'].mean()).reset_index()

In [396]:
# Merge Dataframes
df_text_character_counts_month = df_text_average_character_median_per_month.merge(df_text_average_character_mean_per_month, on='yearmonth', how='left')
df_text_character_counts_month.rename(columns = {'Kommentar_Character_x':'Median Characters','Kommentar_Character_y':'Mittelwert Characters'}, inplace = True)

In [426]:
fig = px.line(df_text_character_counts_month,
              x="yearmonth",
              y=['Median Characters','Mittelwert Characters'],
              color_discrete_sequence=color_discrete_sequence,
              template=template)
fig.update_layout(
    title="Freitextkommentare: Characters-Median & Characters-Mittelwert in Vergleich",
    yaxis_title='Anzahl',
    xaxis_title='Monat',
    width=1200, 
    height=600,
    legend_title="")
fig.update_traces(line_width=2)
fig.update_yaxes(range=(81,200))



fig.show()

pio.write_image(fig,"images/fig-Characters-median-mittelwert.svg") #save fig as svg

In [427]:
fig = px.histogram(
    df_text,
    x="Kommentar_Character",
    range_x = [1,1500],
    title="Freitextkommentare: Häufigkeitsverteilung Länge der Kommentare (Characters)",
    color = "year",
    labels={'Kommentar_Character':'Anzahl Character'},
    color_discrete_sequence=color_discrete_sequence,
    width=1200, 
    height=600,
    template=template)
    
fig.update_layout(yaxis_title="Anzahl Kommentare")

# Overlay both histograms
fig.update_layout(barmode='overlay')

# Reduce opacity to see both histograms
fig.update_traces(opacity=0.4)

fig.show()

pio.write_image(fig,"images/fig-character-hist.svg") #save fig as svg

In [373]:
df_text.describe()

,participant_id,year,month,wime_personal,wime_komfort,wime_sauberkeit,wime_puenktlich,wime_platzangebot,wime_gesamtzuf,wime_preis_leistung,wime_fahrplan,wime_oes_fahrt,S_alter,u_preis,Kommentar_Character,Kommentar_Tokens,Kommentar_Types,Kommentar_TTR
count,61035.000000,61035.000000,61035.000000,22575.000000,48969.000000,49642.000000,49816.000000,50110.000000,52122.000000,57118.000000,60178.000000,47609.000000,60884.000000,53544.000000,61035.000000,61035.000000,61035.000000,60958.000000
mean,360513.030982,2020.353043,6.530024,88.176080,74.256219,76.587254,86.970775,75.475121,78.716921,67.814340,78.235632,89.411666,51.928290,43.779191,173.385074,26.894175,23.981945,94.695901
std,176093.183053,1.181380,3.298886,19.895906,25.702607,23.187087,24.406390,30.283246,23.358262,29.246846,27.668477,16.169346,15.581898,276.715213,181.906399,28.602560,22.564262,6.905334
min,41305.000000,2019.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10.000000,0.000000,0.000000,0.000000,0.000000,33.333333
25%,120151.500000,2019.000000,4.000000,77.777778,55.555556,66.666667,77.777778,55.555556,75.000000,50.000000,66.666667,77.777778,41.000000,8.800000,58.000000,9.000000,8.000000,90.697674
50%,406700.000000,2020.000000,7.000000,100.000000,75.000000,77.777778,100.000000,88.888889,77.777778,75.000000,88.888889,100.000000,54.000000,17.000000,119.000000,18.000000,17.000000,100.000000
75%,501985.500000,2021.000000,9.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,64.000000,30.600000,219.000000,34.000000,32.000000,100.000000
max,587992.000000,2022.000000,12.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,98.000000,6300.000000,2047.000000,347.000000,230.000000,100.000000


In [374]:
df_text.quantile(0.9) 

/var/folders/zg/d4glzd_11n11pmq9m6l8yksm0000gn/T/ipykernel_22157/4176799490.py:1: FutureWarning:

The default value of numeric_only in DataFrame.quantile is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.



participant_id         550512.6
year                     2022.0
month                      11.0
wime_personal             100.0
wime_komfort              100.0
wime_sauberkeit           100.0
wime_puenktlich           100.0
wime_platzangebot         100.0
wime_gesamtzuf            100.0
wime_preis_leistung       100.0
wime_fahrplan             100.0
wime_oes_fahrt            100.0
S_alter                    71.0
u_preis                    49.0
Kommentar_Character       380.0
Kommentar_Tokens           60.0
Kommentar_Types            52.0
Kommentar_TTR             100.0
Name: 0.9, dtype: float64

In [375]:
#df_text[df_text.year == 2019][df_text.Kommentar_Character == 199]

In [376]:
df_text

,participant_id,u_date,year,month,Kommentar,wime_personal,wime_komfort,wime_sauberkeit,wime_puenktlich,wime_platzangebot,...,fg_zielort_uic,fg_startort,fg_zielort,ft_startort,ft_zielort,yearmonth,Kommentar_Character,Kommentar_Tokens,Kommentar_Types,Kommentar_TTR
225817,583530,2022-10-30,2022,10,Klimaanlage / Heizung wechselte leider ständig...,NaN,75.000000,100.000000,100.000000,75.000000,...,8506000,Lodano,Winterthur,Bellinzona,Zürich HB,2022-10-01,63,10,10,100.000000
226048,583862,2022-10-30,2022,10,Am Morgen des gleichen Tages fuhren wir mit ei...,NaN,NaN,NaN,NaN,NaN,...,NaN,Chur,"Rafz, Friedhof",Chur,"Rafz, Friedhof",2022-10-01,376,57,56,98.245614
226052,583866,2022-10-30,2022,10,Wir haben 4 Veloplätze reserviert. Die Zuteilu...,NaN,NaN,NaN,NaN,NaN,...,NaN,Lugano,Zürich HB,Lugano,Zürich HB,2022-10-01,342,54,51,94.444444
226062,583878,2022-10-30,2022,10,"Anzeige, ob viel Menschen diesen Zug benützen!...",NaN,NaN,NaN,NaN,NaN,...,NaN,Winterthur,Pilatus Kulm,Winterthur,Pilatus Kulm,2022-10-01,111,19,18,94.736842
226063,583879,2022-10-30,2022,10,Zug war voll,NaN,NaN,NaN,NaN,NaN,...,NaN,Zürich HB,Basel SBB,Zürich HB,Basel SBB,2022-10-01,12,3,3,100.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
892,51631,2019-01-02,2019,1,in den IR in der 1. klasse bequemere sitze ein...,100.000000,44.444444,100.000000,100.000000,100.000000,...,8508100,Wallisellen,Langenthal,Zürich Altstetten,Langenthal,2019-01-01,51,10,9,90.000000
890,51628,2019-01-02,2019,1,"Retourfahrten, d.h. Luzern - Wohlen/Bremgarten...",77.777778,77.777778,66.666667,100.000000,66.666667,...,8505000,Bremgarten Isenlauf,Luzern,Rotkreuz,Luzern,2019-01-01,104,15,14,93.333333
1023,51805,2019-01-02,2019,1,Die signalisation des gleises 50 in Zofingen w...,NaN,77.777778,100.000000,100.000000,100.000000,...,8502196,Dagmersellen,Unterentfelden Post,Dagmersellen,Zofingen,2019-01-01,138,20,19,95.000000
1018,51799,2019-01-02,2019,1,die lüftung (frischluft) der klimatisierten sb...,NaN,66.666667,44.444444,66.666667,66.666667,...,8506121,Frauenfeld,Romanshorn,Frauenfeld,Weinfelden,2019-01-01,150,20,19,95.000000


- Median of 122 Characters per Comment (140 original Twitter length)
- Median of 19 Tokens per comment.
- Median TTR of 98.

In [377]:
satisfaction_codes = config[config['vartype'] == 'satisfaction']['fragecode'] #loading satisfaction codes from config
df_text['rating_selected_average'] = df_text[satisfaction_codes].mean(axis=1) #calculate mean ratings 

satisfaction_category = pd.cut(df_text['rating_selected_average'],bins=[0,25,50,75,100],labels=["<25 Punkte","25-50 Punkte","50-75 Punkte","75-100 Punkte"])
df_text.insert(49,"rating_selected_average_group",satisfaction_category)

satisfaction_category_spontan = pd.cut(df_text['wime_gesamtzuf'],bins=[0,25,50,75,100],labels=["<25 Punkte","25-50 Punkte","50-75 Punkte","75-100 Punkte"])
df_text.insert(50,"rating_wime_gesamtzuf_group",satisfaction_category_spontan)

In [380]:
# x and y given as array_like objects
fig = px.scatter(
    df_text,
    x='Kommentar_Character', 
    y='Kommentar_TTR',
    opacity = 1,
    color_discrete_sequence =color_discrete_sequence,
    color = "rating_selected_average_group",
    marginal_x='box',
    #color_continuous_scale=[(0,'red'), (0.75, 'orange'), (1,'green')]
                 )

fig.update_layout(
    title="Kommentarlänge und Type-Token Ratio im Vergleich mit Zufriedenheitsbewertung",
    yaxis_title='TTR in %',
    xaxis_title='Kommentarlänge in Character',
    width=1200, 
    height=600,
    legend_title="")

#fig.update_yaxes(range=(55,105))

fig.show()

pio.write_image(fig,"images/fig-Characters-TTR_scatter.svg") #save fig as svg

In [343]:
# Pandas Profiling TextData
profile = ProfileReport(
    df_text[
        [
            #"participant_id",
            #"u_date",
            "Kommentar",
            "Kommentar_Character",
            "Kommentar_TTR",
            "Kommentar_Tokens",
            "Kommentar_Types",
            'rating_selected_average',
        ]
    ],
    title="TextData",
    lazy=False,
    dark_mode=True,
)

profile.to_file("html/ProfilingTextData.html")


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]/Users/dominik/Documents/Masterarbeit-Code/nlp-satisfaction/venv/lib/python3.9/site-packages/multimethod/__init__.py:315: FutureWarning:

iteritems is deprecated and will be removed in a future version. Use .items instead.

Summarize dataset:  47%|████▋     | 9/19 [00:04<00:04,  2.03it/s, Calculate spearman correlation]           /Users/dominik/Documents/Masterarbeit-Code/nlp-satisfaction/venv/lib/python3.9/site-packages/multimethod/__init__.py:315: FutureWarning:

The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 91.15it/s]


### Satisfaction Data

In [155]:
satisfaction_codes = config[config['vartype'] == 'satisfaction']['fragecode']

In [156]:
# Pandas Profiling Satisfaction Data
profile = ProfileReport(
    df[
        satisfaction_codes
    ],
    title="SatisfactionData",
    lazy=False,
    dark_mode=True,
)

profile.to_file("html/ProfilingSatisfactionData.html")


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]/Users/dominik/Documents/Masterarbeit-Code/nlp-satisfaction/venv/lib/python3.9/site-packages/multimethod/__init__.py:315: FutureWarning:

iteritems is deprecated and will be removed in a future version. Use .items instead.

Summarize dataset:  50%|█████     | 11/22 [00:01<00:01,  7.24it/s, Calculate auto correlation]          


KeyboardInterrupt: 

In [169]:
df_satisfaction = pd.melt(df, id_vars='yearmonth', value_vars=satisfaction_codes) #wide to long and only keep satisfaction questions
df_satisfaction = df_satisfaction.groupby(['yearmonth', 'variable'])[['value']].mean().reset_index() #groupy monthyear and variable

df_satisfaction_average = df_satisfaction.groupby(['yearmonth'])[['value']].mean().reset_index() #groupy monthyear to get overall average
df_satisfaction_average.insert(1,'variable',"Durchschnitt")

df_satisfaction = pd.concat([df_satisfaction,df_satisfaction_average],ignore_index=True).sort_values('yearmonth')

In [170]:
df_satisfaction

,yearmonth,variable,value
0,2019-01-01,wime_fahrplan,80.883316
8,2019-01-01,wime_sauberkeit,78.147641
7,2019-01-01,wime_puenktlich,89.092469
6,2019-01-01,wime_preis_leistung,72.451744
5,2019-01-01,wime_platzangebot,79.529696
...,...,...,...
392,2022-10-01,wime_platzangebot,76.046301
393,2022-10-01,wime_preis_leistung,73.876311
394,2022-10-01,wime_puenktlich,88.249476
395,2022-10-01,wime_sauberkeit,77.739966


In [434]:
fig = px.line(
    df_satisfaction,
    x="yearmonth",
    y="value",
    color="variable",
    color_discrete_sequence=color_discrete_sequence,
    template=template,
)

fig.update_layout(
    title="Zufriedenheit mit verschiedenen Aspekten im zeitlichen Verlauf",
    yaxis_title="Zufriedenheit",
    xaxis_title="",
    width=1200, 
    height=600,
    legend_title="",
)

fig.update_traces(line_width=2)
fig.update_yaxes(range=(62, 100))
fig.add_vrect(
    x0="2020-03-25",
    x1="2022-03-01",
    line_width=0,
    fillcolor="#0B1F26",
    opacity=0.1,
    annotation_text="COVID-19 Massnahmen",
    annotation_position="top right",
    annotation_font_size=10,
    annotation_font_color="#0B1F26",
)

fig.add_vrect(
    x0="2020-04-01",
    x1="2020-05-30",
    line_width=0,
    fillcolor="#0B1F26",
    opacity=0.2,
    annotation_text="Pause",
    annotation_position="top left",
    annotation_font_size=10,
    annotation_font_color="#0B1F26",
)



line_width = 2

fig.update_traces(patch={"line": {"dash": 'dot'}}, selector={"legendgroup": "Durchschnitt"})

fig.show()

pio.write_image(fig, "images/fig-satisfaction_zeitreihe.svg")  # save fig as png


### MetaData

In [25]:
meta_codes = config[config['vartype'] == 'meta']['fragecode']

In [63]:
# Pandas Profiling MetaData
profile = ProfileReport(
    df[
        [meta_codes]
    ],
    title="MetaData",
    lazy=False,
    dark_mode=True,
)

profile.to_file("html/ProfilingMetaData.html")


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]/Users/dominik/Documents/nlp-satisfaction/venv/lib/python3.9/site-packages/multimethod/__init__.py:315: FutureWarning:

iteritems is deprecated and will be removed in a future version. Use .items instead.

Summarize dataset:  75%|███████▌  | 33/44 [00:04<00:01,  7.80it/s, Calculate auto correlation]       /Users/dominik/Documents/nlp-satisfaction/venv/lib/python3.9/site-packages/scipy/stats/_stats_py.py:112: RuntimeWarning:

The input array could not be properly checked for nan values. nan values will be ignored.

/Users/dominik/Documents/nlp-satisfaction/venv/lib/python3.9/site-packages/scipy/stats/_stats_py.py:112: RuntimeWarning:

The input array could not be properly checked for nan values. nan values will be ignored.

/Users/dominik/Documents/nlp-satisfaction/venv/lib/python3.9/site-packages/scipy/stats/_stats_py.py:112: RuntimeWarning:

The input array could not be properly checked for nan values. nan values will be ignored.